<a href="https://colab.research.google.com/github/hamednasr/Computer-Vision-with-Python/blob/main/body_move_detection_for_game(nich).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
import numpy as np
from matplotlib import pyplot as plt
import cv2
import os
import time

In [ ]:
interpreter = tf.lite.Interpreter(model_path='lite-model_movenet_singlepose_lightning_3.tflite')
interpreter.allocate_tensors()
input_size = 192
# input_size = 256

In [ ]:
EDGES = {
    (0, 1): 'm',
    (0, 2): 'c',
    (1, 3): 'm',
    (2, 4): 'c',
    (0, 5): 'm',
    (0, 6): 'c',
    (5, 7): 'm',
    (7, 9): 'm',
    (6, 8): 'c',
    (8, 10): 'c',
    (5, 6): 'y',
    (5, 11): 'm',
    (6, 12): 'c',
    (11, 12): 'y',
    (11, 13): 'm',
    (13, 15): 'm',
    (12, 14): 'c',
    (14, 16): 'c'
}

In [ ]:
def draw_keypoints(frame, keypoints, confidence_threshold):
    y, x, c = frame.shape
    shaped = np.squeeze(np.multiply(keypoints, [y,x,1]))
    
    for kp in shaped:
        ky, kx, kp_conf = kp
        if kp_conf > confidence_threshold:
            cv2.circle(frame, (int(kx), int(ky)), 4, (0,255,0), -1) 

In [ ]:
def draw_connections(frame, keypoints, edges, confidence_threshold):
    y, x, c = frame.shape
    shaped = np.squeeze(np.multiply(keypoints, [y,x,1]))
    
    for edge, color in edges.items():
        p1, p2 = edge
        y1, x1, c1 = shaped[p1]
        y2, x2, c2 = shaped[p2]
        
        if (c1 > confidence_threshold) & (c2 > confidence_threshold):      
            cv2.line(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0,0,255), 2)

In [ ]:
def movenet_keypoints(image, input_size=192):
    
    # Reshape image
    img = tf.image.resize_with_pad(np.expand_dims(image, axis=0), input_size,input_size)
    input_image = tf.cast(img, dtype=tf.float32)

    # Setup input and output 
    input_details = interpreter.get_input_details()
    output_details = interpreter.get_output_details()

    # Make predictions 
    interpreter.set_tensor(input_details[0]['index'], np.array(input_image))
    interpreter.invoke()
    keypoints_with_scores = interpreter.get_tensor(output_details[0]['index'])
    
    
    return keypoints_with_scores   

# draw keypoints on cam video:

In [ ]:
cap = cv2.VideoCapture(0)
while cap.isOpened():
    ret, frame = cap.read()
    
    # Reshape image
    
    img = tf.image.resize_with_pad(np.expand_dims(frame, axis=0), input_size,input_size)
    input_image = tf.cast(img, dtype=tf.float32)
    
    # Setup input and output 
    input_details = interpreter.get_input_details()
    output_details = interpreter.get_output_details()
    
    # Make predictions 
    interpreter.set_tensor(input_details[0]['index'], np.array(input_image))
    interpreter.invoke()
    keypoints_with_scores = interpreter.get_tensor(output_details[0]['index'])
#     print(np.squeeze(keypoints_with_scores).flatten())
    # Rendering 
    draw_connections(frame, keypoints_with_scores, EDGES, 0.2)
    draw_keypoints(frame, keypoints_with_scores, 0.2)
    
    cv2.imshow('MoveNet Lightning', frame)
    
    if cv2.waitKey(10) & 0xFF==ord('q'):
        break
        
cap.release()
cv2.destroyAllWindows()

# record data:

## Jump:

In [ ]:
data_path = 'C:\\Users\\david\\Google Drive\\Colab Notebooks\\Computer Vision\\bodymove_data'
no_videos = 6
sequences_length = 20

In [ ]:
for video in range(no_videos):
    try:
        os.makedirs(os.path.join(data_path,'jump',str(video)))  
    except:
        pass        

In [ ]:
cap = cv2.VideoCapture(0)
    
for video in range(no_videos):
    for frame_num in range(sequences_length):

        ret, frame = cap.read()

        keypoints_with_scores = movenet_keypoints(frame, input_size=192)
        draw_keypoints(frame, keypoints_with_scores, 0.2)
        draw_connections(frame, keypoints_with_scores, EDGES, 0.2)

        if frame_num==0:
            cv2.putText(frame,'COLLECTION START for JUMP',(150,200),
                        cv2.FONT_HERSHEY_COMPLEX,1,(0,0,255),4,cv2.LINE_AA)
            cv2.putText(frame,f'collecting frames for jump video numer {video}',(15,12),
                        cv2.FONT_HERSHEY_COMPLEX,.5,(0,255,0),1,cv2.LINE_AA)
            cv2.imshow('Webcam Feed', frame)
            cv2.waitKey(2000)

        else:
            cv2.putText(frame,f'collecting frames for jump video numer {video}',(15,12),
            cv2.FONT_HERSHEY_COMPLEX,0.5,(0,255,0),1,cv2.LINE_AA)

            cv2.imshow('Webcam Feed', frame)
            
            
        keypoints = np.squeeze(keypoints_with_scores).flatten()

        npy_path = os.path.join(data_path,'jump',str(video),str(frame_num))
        np.save(npy_path,keypoints)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()

## Crouch:

In [ ]:
data_path = 'C:\\Users\\david\\Google Drive\\Colab Notebooks\\Computer Vision\\bodymove_data'
no_videos = 60
sequences_length = 20

In [ ]:
for video in range(no_videos):
    try:
        os.makedirs(os.path.join(data_path,'crouch',str(video)))  
    except:
        pass        

In [ ]:
cap = cv2.VideoCapture(0)
    
for video in range(no_videos):
    for frame_num in range(sequences_length):

        ret, frame = cap.read()

        keypoints_with_scores = movenet_keypoints(frame, input_size=192)
        draw_keypoints(frame, keypoints_with_scores, 0.2)
        draw_connections(frame, keypoints_with_scores, EDGES, 0.2)

        if frame_num==0:
            cv2.putText(frame,'COLLECTION START for CROUCH',(150,200),
                        cv2.FONT_HERSHEY_COMPLEX,1,(0,0,255),4,cv2.LINE_AA)
            cv2.putText(frame,f'collecting frames for crouch video numer {video}',(15,12),
                        cv2.FONT_HERSHEY_COMPLEX,.5,(0,255,0),1,cv2.LINE_AA)
            cv2.imshow('Webcam Feed', frame)
            cv2.waitKey(2000)

        else:
            cv2.putText(frame,f'collecting frames for crouch video numer {video}',(15,12),
            cv2.FONT_HERSHEY_COMPLEX,0.5,(0,255,0),1,cv2.LINE_AA)

            cv2.imshow('Webcam Feed', frame)
            
            
        keypoints = np.squeeze(keypoints_with_scores).flatten()

        npy_path = os.path.join(data_path,'crouch',str(video),str(frame_num))
        np.save(npy_path,keypoints)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()

## right jump:

In [ ]:
data_path = 'C:\\Users\\david\\Google Drive\\Colab Notebooks\\Computer Vision\\bodymove_data'
no_videos = 60
sequences_length = 20

In [ ]:
for video in range(no_videos):
    try:
        os.makedirs(os.path.join(data_path,'right_jump',str(video)))  
    except:
        pass        

In [ ]:
cap = cv2.VideoCapture(0)
    
for video in range(no_videos):
    for frame_num in range(sequences_length):

        ret, frame = cap.read()

        keypoints_with_scores = movenet_keypoints(frame, input_size=192)
        draw_keypoints(frame, keypoints_with_scores, 0.2)
        draw_connections(frame, keypoints_with_scores, EDGES, 0.2)

        if frame_num==0:
            cv2.putText(frame,'COLLECTION START for RIGHT_JUMP',(150,200),
                        cv2.FONT_HERSHEY_COMPLEX,1,(0,0,255),4,cv2.LINE_AA)
            cv2.putText(frame,f'collecting frames for right_jump video numer {video}',(15,12),
                        cv2.FONT_HERSHEY_COMPLEX,.5,(0,255,0),1,cv2.LINE_AA)
            cv2.imshow('Webcam Feed', frame)
            cv2.waitKey(2000)

        else:
            cv2.putText(frame,f'collecting frames for right_jump video numer {video}',(15,12),
            cv2.FONT_HERSHEY_COMPLEX,0.5,(0,255,0),1,cv2.LINE_AA)

            cv2.imshow('Webcam Feed', frame)
            
            
        keypoints = np.squeeze(keypoints_with_scores).flatten()

        npy_path = os.path.join(data_path,'right_jump',str(video),str(frame_num))
        np.save(npy_path,keypoints)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()

## left jump:

In [ ]:
data_path = 'C:\\Users\\david\\Google Drive\\Colab Notebooks\\Computer Vision\\bodymove_data'
no_videos = 60
sequences_length = 20

In [ ]:
for video in range(no_videos):
    try:
        os.makedirs(os.path.join(data_path,'left_jump',str(video)))  
    except:
        pass        

In [ ]:
cap = cv2.VideoCapture(0)
    
for video in range(no_videos):
    for frame_num in range(sequences_length):

        ret, frame = cap.read()

        keypoints_with_scores = movenet_keypoints(frame, input_size=192)
        draw_keypoints(frame, keypoints_with_scores, 0.2)
        draw_connections(frame, keypoints_with_scores, EDGES, 0.2)

        if frame_num==0:
            cv2.putText(frame,'COLLECTION START for LEFT_JUMP',(150,200),
                        cv2.FONT_HERSHEY_COMPLEX,1,(0,0,255),4,cv2.LINE_AA)
            cv2.putText(frame,f'collecting frames for left_jump video numer {video}',(15,12),
                        cv2.FONT_HERSHEY_COMPLEX,.5,(0,255,0),1,cv2.LINE_AA)
            cv2.imshow('Webcam Feed', frame)
            cv2.waitKey(2000)

        else:
            cv2.putText(frame,f'collecting frames for left_jump video numer {video}',(15,12),
            cv2.FONT_HERSHEY_COMPLEX,0.5,(0,255,0),1,cv2.LINE_AA)

            cv2.imshow('Webcam Feed', frame)
            
            
        keypoints = np.squeeze(keypoints_with_scores).flatten()

        npy_path = os.path.join(data_path,'left_jump',str(video),str(frame_num))
        np.save(npy_path,keypoints)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()